# Testscript.ipynb

A Jupyter notebook for testing the `ArchetypeBuildingWeather.py` module.
Doubles as a *"tutorial"* of sorts.

In [ ]:
## Import module

import archetypebuildingweather as abw

In [ ]:
## Initialize input parameters

shapefile_path = "data/FI.shp"
raster_path = "data/popu01clcv5_epsg4326.tif"
#weights = {'KU005':0.24, 'KU010':0.73, 'KU009':0.03,} # Creating random weights later on.
weather_start = "2010-06"
weather_end = "2010-06"

In [ ]:
## Try creating a `Shapefile` and inspect the uniform raster

import matplotlib.pyplot as plt
ec = (0,0,0,1) # Edge color for the vector plots (black)
fc = (0,0,0,0) # Face color for the vector plots (transparent)
lw = 0.2 # Line width for the vector plots

shp = abw.Shapefile(shapefile_path)

f = abw.plot_layout(shp, shp.raster, dpi=100, title="Shapefile")

In [ ]:
## Create a full-Finland random weights for testing.

import random

weights = {lid:random.uniform(0,1) for lid in shp.data.index}

In [ ]:
## Try creating the cutout

cutout = abw.prepare_cutout(shp, weather_start, weather_end)

In [ ]:
## Try preparing the `layout` raster and inspect nonzero data.

raster = abw.prepare_layout(shp, weights, raster_path, method='mean')
f = abw.plot_layout(shp, raster, title="Raw weights")
f.savefig("figs/weights.png", dpi=600)

In [ ]:
## Check the coarsened raster without reindexing.

rst = abw.coarsen_raster(raster, cutout.data)
f = abw.plot_layout(shp, rst, title="Coarse weights")

In [ ]:
## Match the layout to the atlite cutout and inspect nonzero data.

layout = abw.match_layout(shp, raster, cutout, reindex_method="nearest")
f = abw.plot_layout(shp, layout, title="PyPSA/atlite layout")
f.savefig("figs/layout.png", dpi=600)

In [ ]:
## Process weather data for the cutout.

ambient_temperature, diffuse_irradiation, direct_irradiation = abw.process_weather(cutout, layout)

In [ ]:
# Plot temperature

ambient_temperature.plot()

In [ ]:
# Plot diffuse irradiation

diffuse_irradiation.plot()

In [ ]:
# Plot direct irradiation

import pandas as pd

pd.DataFrame(direct_irradiation).plot()

In [ ]:
## Now do it all again using the convenience function:

ambient_temperature, diffuse_irradiation, direct_irradiation = abw.aggregate_weather(shapefile_path, weights, weather_start, weather_end, raster_path)
pd.DataFrame(ambient_temperature).plot()
pd.DataFrame(diffuse_irradiation).plot()
pd.DataFrame(direct_irradiation).plot()